<a href="https://colab.research.google.com/github/Alinehbg/CRP/blob/Anmol_Trial/Extract%20Names/Extract_Wiki_Data_names_try_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
names = pd.read_excel('/content/drive/MyDrive/CRP/Datasets/dataset trial.xlsx')
names['Name'] = names.apply (lambda x: x['Name'].replace(".","") , axis = 1)
names['Name_Length'] = names.apply(lambda x: len(x['Name'].split(" ")), axis = 1)
names['First_Name_Length'] = names.apply(lambda x: len(x['Name'].split(" ")[0]), axis = 1)
names = names.loc[names['Name_Length'] == 2]
names = names.loc[names['First_Name_Length'] > 1]
names['Name'] = names.apply(lambda x: x['Name'].title(), axis = 1 )
names = names[["Name"]]
names_list = list(names['Name'])
names_list = list(set(names_list))
names.head()

,Name
1,Egils Bojrs
4,Vjaeslavs Fandus
5,Johan Frber
10,Ioannis Fasilis
11,Adrien Fauchier-Magnan


In [6]:
first_names = [i.split(" ")[0] for i in names_list ]
last_names = [i.split(" ")[1] for i in names_list ]

In [7]:
first_names = list(set(first_names))
last_names = list(set(last_names))

In [8]:
len(first_names)
len(last_names)

5

#### functions 

In [10]:
def get_itemlabel(query): 
  name_list = []
  for x in query["results"]["bindings"]: 
    name_list.append(x["itemLabel"]["value"])
  return name_list

def retry(data): 
  response = requests.post('https://query.wikidata.org/sparql?format=json', data=data)
  while response.status_code == 429:
    retry_after = response.headers.get("Retry-After")
    if retry_after:
      # If the value of "Retry-After" is a number, wait that many seconds
      if retry_after.isdigit():
        time.sleep(int(retry_after))
        # If the value of "Retry-After" is a date, parse it and wait until that time
      else:
        retry_time = datetime.datetime.strptime(retry_after, "%a, %d %b %Y %H:%M:%S GMT")
        if retry_time > datetime.datetime.now():
          time.sleep((retry_time - datetime.datetime.now()).total_seconds())
    # Try the request again
    response = requests.post('https://query.wikidata.org/sparql?format=json', data=data)
  res = response.json()
  return res

def find_fname_id(name): 
  find_fn_id_query_male = f'''SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    ?item rdfs:label "{name}"@en;
          wdt:P31 wd:Q12308941.
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
  }}
  '''
  find_fn_id_query_female = f'''SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    ?item rdfs:label "{name}"@en;
          wdt:P31 wd:Q12308941.
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
  }}
  '''
  data = {'query': find_fn_id_query_male}
  res = retry(data)
  if res['results']['bindings'] == []: 
    data = {'query': find_fn_id_query_female}
    res = retry(data)
  try:
    id = res['results']['bindings'][0]['item']['value'].split('/')[-1]
  except IndexError:
    #print(f"Cannot find first name id for {name}")
    id = []
  return id

def find_lname_id(name): 
  find_fn_id_query = f'''SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    ?item rdfs:label "{name}"@en;
          wdt:P31 wd:Q101352.
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
  }}
  '''
  data = {'query': find_fn_id_query}
  res = retry(data)
  try:
      id = res['results']['bindings'][0]['item']['value'].split('/')[-1]
  except IndexError:
      #print(f"Cannot find last name id for {name}")  
      id = []  
  return id

def find_similar_from_id(id): 
  #find similar names from the ID just found for the target name
  id = id
  find_similar_query = f"""
  SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    {{
      SELECT DISTINCT ?item WHERE {{
        ?item p:P460 ?statement0.
        ?statement0 (ps:P460/(wdt:P279*)) wd:{id}.
      }}
    }}
  }}
  """
  data = {'query': find_similar_query}
  res = retry(data)
  similar_name_list = get_itemlabel(res)
  return similar_name_list

def similar_full_names(name): 
  f_name, l_name = name.split(' ')

  f_name_id = find_fname_id(f_name)
  l_name_id = find_lname_id(l_name)

  if f_name_id == [] and l_name_id != []: 
    fname_similar = []
    lname_similar = find_similar_from_id(l_name_id)
    fname_list = []
    lname_list = [f_name + ' ' + sim_lname for sim_lname in lname_similar]
  elif l_name_id == [] and f_name_id != []: 
    lname_similar = []
    fname_similar = find_similar_from_id(f_name_id)
    lname_list = []
    fname_list = [sim_fname + ' ' + l_name for sim_fname in fname_similar]
  elif f_name_id == [] and l_name_id == []: 
    fname_list = []
    lname_list = []
  else: 
    fname_similar = find_similar_from_id(f_name_id)
    lname_similar = find_similar_from_id(l_name_id)

    fname_list = [sim_fname + ' ' + l_name for sim_fname in fname_similar]
    lname_list = [f_name + ' ' + sim_lname for sim_lname in lname_similar]
  
  similar_full_names = fname_list + lname_list
  pair = {name: similar_full_names}
  return pair

def Merge(dict1, dict2):
  return(dict1.update(dict2))

####requests and the rest

In [16]:
#loop through all the names in the list and add key(target name) value(list of similar names) pairs to dictionary 
first_name_dict = {}

for f_name in first_names: 
    
    f_name_id = find_fname_id(f_name)
    
    if f_name_id == []: 
      fname_list = [f_name]

    elif f_name_id != []: 
      fname_similar = find_similar_from_id(f_name_id)
      fname_list = [sim_fname for sim_fname in fname_similar]
      fname_list.append(f_name)
    temp = {f_name: fname_list}
    Merge(first_name_dict, temp)

print(len(first_name_dict))
print(first_name_dict)

5
{'Egils': ['Egil', 'Egīls', 'Egils'], 'Ioannis': ['Juan', 'Ivo', 'Jack', 'Ivan', 'Ion', 'Gian', 'Giovanni', 'Hans', 'Jógvan', 'Johnny', 'Johannes', 'Ieuan', 'John', 'Honza', 'Jean', 'Siôn', 'Yannis', 'Johan', 'Jan', 'Johann', 'Jon', 'Evan', 'Joan', 'Ioannes', 'Ioann', 'Yvan', 'Hannes', 'Joannes', 'João', 'Xoán', 'Ian', 'Ivashka', 'Gianni', 'Sean', 'Ioan', 'Ivan', 'Jehan', 'Iwan', 'Jaan', 'Yanka', 'Jón', 'Jóhan', 'Janka', 'Hanns', 'Iwo', 'Giannis', 'Ivan', 'Jóhann', 'Yanni', 'János', 'Yann', 'Jóhannes', 'Erwann', 'Seán', 'Yannis', 'Ivano', 'Ifan', 'Johannès', 'Yiannis', 'Ioánnis', 'Iván', 'Ionuț', 'Giovan', 'Joanis', 'Ioannis', 'Giuàn', 'Jhoao', 'Ioannis'], 'Vjaeslavs': ['Vjaeslavs'], 'Johan': ['Juan', 'Ivo', 'Jack', 'Ivan', 'Ion', 'Gian', 'Giovanni', 'Hans', 'Jógvan', 'Johnny', 'Johannes', 'Ieuan', 'John', 'Honza', 'Jean', 'Siôn', 'Yannis', 'Jan', 'Johann', 'Jussi', 'Jon', 'Evan', 'Joan', 'Ioannes', 'Ioann', 'Yvan', 'Hannes', 'Joannes', 'João', 'Xoán', 'Ian', 'Ivashka', 'Gianni', 'Se

In [17]:
last_name_dict = {}

for l_name in last_names: 
    
    l_name_id = find_lname_id(l_name)
    
    if l_name_id == []: 
      lname_list = [l_name]

    elif l_name_id != []: 
      lname_similar = find_similar_from_id(l_name_id)
      lname_list = [sim_lname for sim_lname in lname_similar]
      lname_list.append(l_name)
    
    temp = {l_name: lname_list}
    Merge(last_name_dict, temp)

print(len(last_name_dict))
print(last_name_dict)

5
{'Frber': ['Frber'], 'Fasilis': ['Fasilis'], 'Fandus': ['Fandus'], 'Bojrs': ['Bojrs'], 'Fauchier-Magnan': ['Fauchier-Magnan']}


In [20]:
import pandas as pd

first_name_df = pd.DataFrame()

Left = []
Right = []
Match = []

Names = first_names

for name in Names:
  temp = Names
  non_names = temp.remove(name)

  for alternate in first_name_dict.get(name):
    Left.append(name)
    Right.append(alternate)
    Match.append(1)
  
  for non_name in temp:
    Left.append(name)
    Right.append(non_name)
    Match.append(0)
    for non_alternate in first_name_dict.get(non_name):
      Left.append(name)
      Right.append(non_alternate)
      Match.append(0)

#To check
assert len(Left) == len(Right)
assert len(Left) == len(Match)

first_name_df['Target_Name'] = Left
first_name_df['Alternate_Name'] = Right
first_name_df['Match'] = Match

first_name_df.head(200)

,Target_Name,Alternate_Name,Match
0,Egils,Egil,1
1,Egils,Egīls,1
2,Egils,Egils,1
3,Egils,Ioannis,0
4,Egils,Juan,0
...,...,...,...
195,Vjaeslavs,Ivashka,0
196,Vjaeslavs,Gianni,0
197,Vjaeslavs,Sean,0
198,Vjaeslavs,Ioan,0


In [21]:
import pandas as pd

last_name_df = pd.DataFrame()

Left = []
Right = []
Match = []

Names = last_names

for name in Names:
  temp = Names
  non_names = temp.remove(name)

  for alternate in last_name_dict.get(name):
    Left.append(name)
    Right.append(alternate)
    Match.append(1)
  
  for non_name in temp:
    Left.append(name)
    Right.append(non_name)
    Match.append(0)
    for non_alternate in last_name_dict.get(non_name):
      Left.append(name)
      Right.append(non_alternate)
      Match.append(0)

#To check
assert len(Left) == len(Right)
assert len(Left) == len(Match)

last_name_df['Target_Name'] = Left
last_name_df['Alternate_Name'] = Right
last_name_df['Match'] = Match

last_name_df.head(200)

,Target_Name,Alternate_Name,Match
0,Frber,Frber,1
1,Frber,Fasilis,0
2,Frber,Fasilis,0
3,Frber,Fandus,0
4,Frber,Fandus,0
5,Frber,Bojrs,0
6,Frber,Bojrs,0
7,Frber,Fauchier-Magnan,0
8,Frber,Fauchier-Magnan,0
9,Fandus,Fandus,1
